In [32]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
from utils import load_model, display_flow
model, info = load_model("flownet-ss-first-0607")

In [3]:
video = "JABBAWOCKEEZ"


vidcap = cv2.VideoCapture(f'video/{video}.mp4')
sec = 0


frameRate = 0.5
count=1

tensors = []
FRAMES = 21
EVERY = 10

while True:
    count += 1
    if count % EVERY != 0:
        continue
    if count // EVERY > FRAMES:
        break
        
    if count % 100 == 0:
        print(count)

    success, im = vidcap.read()
    if not success:
        break
    arr = np.transpose(im, (2,0,1))
    tensors.append(arr)
    
data = np.array(tensors)
data = data/ 255
print(data.shape)
t = torch.tensor(data, dtype=torch.float32)
t = torch.nn.functional.interpolate(t, size=(384, 512))
print(t.size())
del data

im1_all = t[:-1, :, :, :]
im2_all = t[1:, :, :, :]
X = torch.concat([im1_all, im2_all], dim=1)
print(X.size())
del im1_all, im2_all

device = torch.device('cpu')
X = X.to(device)
model = model.to(device)

Y = []
BATCH_SIZE = 1
batches = [X[BATCH_SIZE*i:BATCH_SIZE*(i + 1), :, :, :] for i in range(X.size(0)//BATCH_SIZE)]


Y = []

for i, x in enumerate(batches):
    print(f"Batch",i)
    y = model(x)
    Y.append(y)


FLOW_T = torch.cat(Y, dim=0)
FLOW_T.size()

del X, Y

flow = FLOW_T.detach().numpy()
del FLOW

N, C, H, W = flow.shape
zeros = np.zeros((N, 1, H, W))
flow_rgb = np.concatenate([flow, zeros], axis=1)
flow_rgb = np.transpose(flow_rgb, (0, 2, 3, 1))
flow_rgb -= flow_rgb.min()
flow_rgb /= flow_rgb.max()
del flow, zeros
print(flow_rgb.shape)

frames = []
for arr in flow_rgb:
    arr = (arr * 255).astype(np.uint8)
    flow = Image.fromarray(arr)
    frames.append(arr)


100
200
(21, 3, 720, 1280)
torch.Size([21, 3, 384, 512])
torch.Size([20, 6, 384, 512])


In [50]:
def write_video(file_path, frames, fps):
    """
    Writes frames to an mp4 video file
    :param file_path: Path to output video, must end with .mp4
    :param frames: List of PIL.Image objects
    :param fps: Desired frame rate
    """
    
    w, h = frames[0].shape[1:]
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    writer = cv2.VideoWriter(file_path, fourcc, fps, (w, h), True)

    for frame in frames:
        frame = frame * 255
        frame = frame.astype(np.uint8)
        writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    writer.release() 

OUTPATH = f"video/{video}_flow.mp4"
write_video(OUTPATH, flow_rgb, 60)


In [52]:
out = cv2.VideoWriter(OUTPATH,cv2.VideoWriter_fourcc(*'DIVX'), 30, (382, 512))
for i in flow_rgb:
    print(i.shape)
    i = i * 255
    i = i.astype(np.uint8)
    out.write(i)
out.release()

(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)
(384, 512, 3)


In [51]:
flow_rgb.shape

(20, 384, 512, 3)